In [2]:
# Model Building
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# For running this notebook either locally or in colab
import sys

In [3]:
%%capture
# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [4]:
# For encoding categorical data
from category_encoders import OneHotEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# NYC Rent

**GOAL:** Improve our model for predicting NYC rent prices.

**Objectives**

- Do one-hot encoding of categorical features
- Do univariate feature selection
- Use scikit-learn to fit Ridge Regression models

# I. Wrangle Data

Create **wrangle function** for **reproducibility**.

In [5]:
def wrangle(filepath):
  df = pd.read_csv(filepath, parse_dates = ['created'], index_col = 'created')
  
  # Remove outlier observations
  df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
          (df['price'] <= np.percentile(df['price'], 99.5)) & 
          (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
          (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
          (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
          (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

  #df = df.drop(columns=['description', 'display_address', 'street_address'], inplace = True)
  drop_cols = [col for col in df.select_dtypes('object') if df[col].nunique() > 10]
  df.drop(columns = drop_cols, inplace=True)
  return df

df = wrangle(DATA_PATH+'apartments/renthop-nyc.csv')

Feature Engineering and Visualization

In [6]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# II. Split Data

Split **target vector** from **feature matrix**.

In [7]:
target = 'price'
y = df[target]

X = df.drop(columns=target)


Split data into **training** and **test** sets.

(Use data from April & May 2016 to train. Use data from June 2016 to test.)

In [8]:
cutoff = '2016-06-24 07:54:24'
mask = X.index < cutoff
X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask],y.loc[~mask]

# III. Establish Baseline

**Note:** This is a **regression** problem because we're predictiong the continuous value `'price'`.

In [9]:
y_pred = [y_train.mean()] * len(y_train)
MAE = mean_absolute_error(y_train, y_pred)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Baseline MAE: 1200.967607548018


# IV. Build Models
 Before we build our model, we need to **encode** our categorical features
-for high-dimensional data, your have a few options first is **dimensionality reduction** using SelectKBest, 

**Question:** How can we represent *categorical* features numerically so that we can use them to train our model?

We represent the categorical variable by encoding

In [16]:
#instatiate Category coders works better 
transformer = OneHotEncoder(use_cat_names = True)

# Fit the Transformer to the training data
transformer.fit(X_train)

#transform the data
XT_train = transformer.transform(X_train)
XT_test = transformer.transform(X_test)
XT_train.shape

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(46135, 31)

In [33]:
#Working on Another transformer
from sklearn.feature_selection import SelectKBest
transformer_2 = SelectKBest(k = 15) # K is a hyperparameter
transformer_2.fit(XT_train, y_train)

#Step:
XTT_train = transformer_2.transform(XT_train)
XTT_test = transformer_2.transform(XT_test)

**Model 1:** Regular ol' linear regression

In [34]:
#instantaite our predictor

model = LinearRegression()

# fit our model

model.fit(XTT_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Model 2:** Ridge regression (regularization)

In [40]:
model_r = Ridge(alpha=1)
model_r.fit(XT_train, y_train)  # This is the OHE data w/ no SelectKBEst


Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

**Model 3:** `SelectKBest`

In [ ]:
model_lr_skb = ...

# V. Check Metrics

In [32]:
print('LR Training MAE', mean_absolute_error(y_train, model.predict(XTT_train)))
print('LR Test MAE', mean_absolute_error(y_test, model.predict(XTT_test)))


Training MAE 698.821151470562
Test MAE 718.7600583221524


In [39]:
print('Ridge Training MAE', mean_absolute_error(y_train, model_r.predict(XT_train)))
print('Ridge Test MAE', mean_absolute_error(y_test, model_r.predict(XT_test)))


Ridge Training MAE 675.3278434077829
Ridge Test MAE 674.7748818887953


# Communicate results